In [ ]:
!pip install datasets transformers Sentencepiece nltk

In [2]:
# download hugging face datasets

import datasets
from datasets import concatenate_datasets

import sentencepiece
from nltk.translate.bleu_score import corpus_bleu
from tqdm import tqdm
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)




cuda


In [ ]:
train_Data = datasets.load_dataset("wmt16","de-en", split="train[:50000]")
val_Data = datasets.load_dataset("wmt16","de-en", split="validation")
test_Data = datasets.load_dataset("wmt16","de-en", split="test")

## Load the model

In [4]:
model_name = "google-t5/t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
translations_validation = []
translations_testing = []
val_test_Data = concatenate_datasets([val_Data, test_Data])
# testing_data = test_Data
print(device)
model.to(device)

# for data in [validation_data, testing_data]:
for example in tqdm(val_test_Data):
    # Prepend the prefix for translation task
    input_text = "translate English to German: " + example["translation"]["en"]  # Modified here
    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    # Generate translation
    translated_ids = model.generate(input_ids,  max_length=512, num_beams=4, early_stopping=True).to(device)
    # Decode the translated ids
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    # if data == validation_data:
    translations_validation.append(translated_text)
    # else:
        # translations_testing.append(translated_text)

# Calculate BLEU score for validation and testing translations
reference_validation = [example["translation"]["de"] for example in val_test_Data]  # Modified here
# reference_testing = [example["translation"]["de"] for example in testing_data]

bleu_score_validation = corpus_bleu([[ref] for ref in reference_validation], translations_validation)
# bleu_score_testing = corpus_bleu([[ref] for ref in reference_testing], translations_testing)

print("BLEU score for validation set:", bleu_score_validation)
# print("BLEU score for testing set:", bleu_score_testing)


cuda


100%|██████████| 5168/5168 [33:33<00:00,  2.57it/s]


BLEU score for validation set: 0.6863329636040714


In [6]:
meteor = datasets.load_metric("meteor")
meteor_score = meteor.compute(predictions=translations_validation, references=reference_validation)
print("METEOR score for validation set:", meteor_score)

# install nltk


<ipython-input-6-7a545b05f963>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  meteor = datasets.load_metric("meteor")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for meteor contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/meteor/meteor.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


METEOR score for validation set: {'meteor': 0.5742548079549654}


In [ ]:
!!pip install bert_score

In [9]:
from bert_score import score
P, R, F1 = score(translations_validation, reference_validation, lang="en", verbose=True)
print("BERT Score for validation set:", F1.mean().item())

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/160 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/81 [00:00<?, ?it/s]

done in 90.90 seconds, 56.86 sentences/sec
BERT Score for validation set: 0.9165948629379272
